# **World Championship 2021**

## **49er and 49erFX**

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
options = Options()
options.add_argument("--headless")

Check your chrome version searching for chrome://version. If your chrome version is 115 or newer, you can install chromedriver here https://googlechromelabs.github.io/chrome-for-testing/#stable. Else, check it here https://chromedriver.chromium.org/downloads. After downloading, unzip the file, move it to the same folder as this notebook and inform the path in the next cell.

In [ ]:
PATH_TO_CHROMEDRIVER = 'chromedriver-mac-x64/chromedriver'

In [3]:
URL = 'https://49er.org/events/2021-world-championship/#result-49'

driver = Chrome(PATH_TO_CHROMEDRIVER, options=options)
driver.get(URL)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser")

Even though we pass the link to extract only the 49er results, 49erFX will also be extracted. We want both data, so we will keep it.

In [4]:
data = []

for row in soup.find_all('tr'):
    resultados = []
    for cell in row.find_all('td'):
        # if cell contains <strike> tag, add () around it
        if cell.find('strike'):
            cell.string = '(' + cell.string + ')'
        resultados.append(cell.text)
    data.append(resultados)

In [5]:
# turn data into a pandas dataframe
df = pd.DataFrame(data)

# change column names
df.columns = ['Posição Geral', 'Sail Number', 'Nome Competidor', 'Nett', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
              'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Pontuação Total']

# drop Sail Number column
df.drop('Sail Number', axis=1, inplace=True)

In [6]:
# divide dataframe into 49er and 49erFX
df_49er = df.iloc[1:37, :]
df_49erFX = df.iloc[38:60, :]

In [7]:
# drop Q17 from df_49erFX
df_49erFX.drop('Q17', axis=1, inplace=True)

In [8]:
# turn all the Q1, Q2, ..., Q17 columns into rows
df_49er = pd.melt(df_49er, id_vars=['Posição Geral', 'Nome Competidor', 'Nett', 'Pontuação Total'], 
                  value_vars=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
                              'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 
                              'Q13', 'Q14', 'Q15', 'Q16', 'Q17'])

df_49erFX = pd.melt(df_49erFX, id_vars=['Posição Geral', 'Nome Competidor', 'Nett', 'Pontuação Total'],
                    value_vars=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
                                'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 
                                'Q13', 'Q14', 'Q15', 'Q16'])

In [9]:
# rename column 'variable' to 'Flotilha'
df_49er.rename(columns={'variable': 'Flotilha',
                        'value': 'Pontuação Regata'}, inplace=True)

df_49erFX.rename(columns={'variable': 'Flotilha',
                          'value': 'Pontuação Regata'}, inplace=True)

In [10]:
# define functions to rename fleet names
def rename_fleet_49er(x):
    if x == 'Q17':
        x = 'MR'
    else:
        x = 'G'
    return x

def rename_fleet_49erFX(x):
    if x == 'Q16':
        x = 'MR'
    else:
        x = 'G'
    return x

df_49er['Flotilha'] = df_49er['Flotilha'].apply(rename_fleet_49er)
df_49erFX['Flotilha'] = df_49erFX['Flotilha'].apply(rename_fleet_49erFX)

In [11]:
df_49er['Nome Competição'] = ['World Championship 2021'] * len(df_49er)
df_49er['ID Competição'] = [11] * len(df_49er)
df_49er['Classe Vela'] = ['49er'] * len(df_49er)
df_49er['Punição'] = [''] * len(df_49er)

df_49erFX['Nome Competição'] = ['World Championship 2021'] * len(df_49erFX)
df_49erFX['ID Competição'] = [11] * len(df_49erFX)
df_49erFX['Classe Vela'] = ['49erFX'] * len(df_49erFX)
df_49erFX['Punição'] = [''] * len(df_49erFX)

In [12]:
df_49er.head()

,Posição Geral,Nome Competidor,Nett,Pontuação Total,Flotilha,Pontuação Regata,Nome Competição,ID Competição,Classe Vela,Punição
0,1,Bart LAMBRIEXFloris van de WERKEN,92.0,118.0,G,3.0,World Championship 2021,11,49er,
1,2,Tim FISCHERFabian GRAF,94.0,110.0,G,1.0,World Championship 2021,11,49er,
2,3,Frederik RASKJakob PRECHT JENSEN,112.0,135.0,G,4.0,World Championship 2021,11,49er,
3,4,Ian BARROWSHans HENKEN,113.0,135.0,G,9.0,World Championship 2021,11,49er,
4,5,Dominik BUKSAKSzymon WIERZBICKI,118.0,134.0,G,10.0,World Championship 2021,11,49er,


In [13]:
df_49erFX.head()

,Posição Geral,Nome Competidor,Nett,Pontuação Total,Flotilha,Pontuação Regata,Nome Competição,ID Competição,Classe Vela,Punição
0,1,Odile van AANHOLTElise RUYTER,75.0,93.0,G,1.0,World Championship 2021,11,49erFX,
1,2,Helene NÆSSMarie RØNNINGEN,83.0,103.0,G,5.0,World Championship 2021,11,49erFX,
2,3,Martine GRAELKahena KUNZE,94.0,112.0,G,10.0,World Championship 2021,11,49erFX,
3,4,Ronja GRÖNBLOMVeera HOKKA,112.0,134.0,G,8.0,World Championship 2021,11,49erFX,
4,5,Patricia SUÁREZMaria CANTERO IZQUIERDO,121.0,141.0,G,7.0,World Championship 2021,11,49erFX,


In [14]:
column_order = ['Nome Competidor', 'ID Competição', 'Classe Vela', 
                'Pontuação Regata', 'Flotilha', 'Posição Geral', 
                'Punição', 'Pontuação Total', 'Nett', 'Nome Competição']

df_49er = df_49er[column_order]
df_49erFX = df_49erFX[column_order]

In [15]:
df_49er.to_csv('scraped-data/World Championship 2021_49er.csv', index=False)
df_49erFX.to_csv('scraped-data/World Championship 2021_49erFX.csv', index=False)